In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from lobio.lob.limit_order import LimitOrder
from lobio.lob.order_book import OrderBook
from lobio.lob.price_level import PriceLevel, TraderId, Side

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
price_level = PriceLevel(LimitOrder(10, 1, Side.BUY, TraderId.MARKET))

price_level.add_limit_order(LimitOrder(10, 0.5, Side.BUY, TraderId.MARKET))

print(price_level == PriceLevel(LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET)))

price_level.add_limit_order(LimitOrder(10, 0.2, Side.BUY, TraderId.MM))

print(price_level.traders_order == [LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET), LimitOrder(10, 0.2, Side.BUY, TraderId.MM)])
print(price_level.quote == 1.7)

price_level.add_limit_order(LimitOrder(10, 0.15, Side.BUY, TraderId.MARKET))

print(price_level.traders_order == [LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET), 
                                    LimitOrder(10, 0.2, Side.BUY, TraderId.MM), 
                                    LimitOrder(10, 0.15, Side.BUY, TraderId.MARKET)])
print(price_level.quote == 1.85)

True
True
True
True
True


In [131]:
price_level = PriceLevel(LimitOrder(20, 0.5, Side.BUY, TraderId.MARKET))

remain_quote, data = price_level.execute_limit_order(0.2)

print(remain_quote == 0.0)
print(data == {TraderId.MARKET: 0.2})
print(price_level == PriceLevel(LimitOrder(20, 0.3, Side.BUY, TraderId.MARKET)))

price_level.add_limit_order(LimitOrder(20, 0.05, Side.BUY, TraderId.MM))
remain_quote, data = price_level.execute_limit_order(0.31)

print(remain_quote == 0.0)
print(data == {TraderId.MARKET: 0.3, TraderId.MM: 0.01})
print(price_level == PriceLevel(LimitOrder(20, 0.04, Side.BUY, TraderId.MM)))

remain_quote, data = price_level.execute_limit_order(1)

print(remain_quote == 0.96)
print(data == {TraderId.MM: 0.04})
print(price_level.quote == 0 and price_level.traders_order == [])

True
True
True
True
True
True
True
True
True


In [132]:
init_lob = {"lastUpdateId": 1, "bids": [[90, 1]], "asks": [[100, 1]]}

diffs = [[2, [[90, 1]], [[100, 1]]], 
         [3, [[89.99, 0.99]], [[100.01, 3]]], 
         [4, [[94.99, 0.0001], [89.99, -0.01]], [[95, 0.0001], [100.01, -1]]],
         [5, [], [[95.01, 0.3], [95.5, 1], [100, 0.1]]],
         [6, [[87, 0.01], [90, 0.01], [94.99, 1]], []],
         [7, [[90, -1.5], [87, -10]], [[95.01, -0.29], [100, -2.1]]],
         [8, [[89.99, -0.98]], [[95, -0.0001]]],
         [9, [[90.01, 0.51]], [[95, 1]]]]

In [133]:
ob = OrderBook.create_lob_init(init_lob)

In [134]:
diff = diffs[0]

In [135]:
ob.apply_historical_update(diff)

In [136]:
ob.bids == [PriceLevel(LimitOrder(90, 2, 0, 0))]

True